In [1]:
import pandas as pd
import numpy as np
train=pd.read_csv("/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/train.csv")
len(train)

287113

In [2]:
train.isnull().sum()

id            0
article       0
highlights    0
dtype: int64

In [3]:
training=train['article'][:200].replace("'", "")
testing=train['highlights'][:200].replace("'", "")

In [4]:
import tensorflow as tf
tokenizer=tf.keras.preprocessing.text.Tokenizer()

def tokenizenews(random_sentence):
#     tokenizer=tf.keras.preprocessing.text.Tokenizer()
    tokenizer.fit_on_texts([random_sentence])
    sequence = tokenizer.texts_to_sequences([random_sentence])
    sequence_length = 50  
    padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=sequence_length)
    return padded_sequence
training=training.apply(tokenizenews,tokenizer)
testing=testing.apply(tokenizenews,tokenizer)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/tmp/ipykernel_26/1899618636.py:11: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  training=training.apply(tokenizenews,tokenizer)
/tmp/ipykernel_26/1899618636.py:12: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  testing=testing.apply(tokenizenews,tokenizer)


In [5]:
from tensorflow.keras.models import Model
vocab_size = 10000
embedding_dim = 200
encoder_hidden_states=[]
sequence_length = 50
input_sequence = tf.keras.Input(shape=(sequence_length, ))
embedding_layer = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim)(input_sequence)
encoder_lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(1600 , return_sequences=True))(embedding_layer)
encoded_representation = tf.keras.layers.Concatenate()([encoder_lstm[:, -1, :], encoder_lstm[:, 0, :]])
model = tf.keras.Model(inputs=input_sequence, outputs=encoded_representation )
def encoder(array,testarray,model):
    opt = tf.keras.optimizers.Adam(learning_rate=0.01)
    for i in range(len(array)):
        with tf.GradientTape() as tape:
            result = model(array[i],training=True)
            testresult=model(testarray[i],training=True)
            loss = tf.keras.losses.mean_squared_error(result, testresult)
#             print(result.shape,testresult.shape,loss)
            encoder_hidden_states.append(result)
        grads = tape.gradient(loss, model.trainable_variables)
        opt.apply_gradients(zip(grads, model.trainable_variables))
    return encoder_hidden_states

In [6]:
def decoder(encoder_hidden_states,target_tokens,model1):
    loss = 0
    context_vector = None
    decoder_state = None
    sequence_length = 50
    hidden_size = 128
    embedding_dim = 128
    vocab_size = 10000
    context_vector=tf.zeros((128,50 ))
    decoder_state=tf.zeros((128, 50))
    learning_rate=0.001
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
    v = tf.random.normal(shape=(hidden_size,50))
    Wh = tf.random.normal(shape=(hidden_size, 50))
    Ws = tf.random.normal(shape=(hidden_size, 50))
    battn = tf.random.normal(shape=(hidden_size,50))
    weights= tf.random.normal(shape=(100, 1))   
    bias=tf.random.uniform([]) 
    for t in range(len(target_tokens)):
        input_sequence = tf.keras.Input(shape=(sequence_length, ))
        embedding_layer = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim)(input_sequence)
#         LSTM=tf.keras.layers.LSTM(128,return_sequences=True)(embedding_layer)
        model = tf.keras.Model(inputs=input_sequence, outputs=embedding_layer)
        output_embeddings = model.predict(target_tokens[t],verbose=0)
        attention_scores = []
        for i in range(len(encoder_hidden_states)):
            encoder_hidden_states[i] = tf.reshape(encoder_hidden_states[i], (128,50))
            e_t_i = v * tf.nn.tanh(Wh * encoder_hidden_states[i]+ Ws*decoder_state+  battn)
            attention_scores.append(e_t_i)
        attention_distributon=tf.nn.softmax(attention_scores)
        for i in range(len(encoder_hidden_states)):
            context_vector += attention_distributon[i] * encoder_hidden_states[i]
        output_embeddings=tf.reshape(output_embeddings, (128, 50, 1))
        output_embeddings=tf.squeeze(output_embeddings, axis=-1)
        concatenated_input = tf.expand_dims(tf.concat([context_vector,output_embeddings], axis=-1),axis=0)
        output_embeddings1=tf.matmul(concatenated_input,weights)+bias
        vocab_distribution=tf.nn.softmax(output_embeddings1,axis=1)
        highest_prob_index=np.argmax(vocab_distribution)
#         print(highest_prob_index)
        target_word_prob = vocab_distribution[0,highest_prob_index,0]
        timestep_loss = -tf.math.log(target_word_prob)
        decoder_state=(decoder_state+context_vector)/2
        decoder_state = tf.reshape(decoder_state, (128, 50))
#         print(timestep_loss)
        loss += timestep_loss
        meanloss=loss/(t+1)
        #optimization process
        v+=learning_rate*meanloss* tf.nn.tanh(Wh * encoder_hidden_states[i]+ Ws*decoder_state+  battn)
        Ws+=learning_rate*meanloss*decoder_state
        Wh=Wh+learning_rate*meanloss*encoder_hidden_states[i]
        battn+=learning_rate*meanloss
        bias+=learning_rate*meanloss
        weights+=learning_rate*meanloss*weights
        decoder_state=(decoder_state+context_vector)/2
        decoder_state = tf.reshape(decoder_state, (128, 50))
    return [v,Ws,Wh,battn,weights,bias]
#         print(v,Ws,Wh,battn,decoder_state,model1.trainable_variables)
        

In [7]:
encoder_hidden=encoder(training,testing,model)
input_sequence1 = tf.keras.Input(shape=(128, 100))
layer2=tf.keras.layers.Dense(1)(input_sequence1)
model1 = tf.keras.Model(inputs=input_sequence1, outputs=layer2)
v,Ws,Wh,battn,weights,bias=decoder(encoder_hidden,testing,model1)
print(model)

In [8]:
def tests(para,v,Ws,Wh,battn,weights,model,bias,tokenizer,length=50):
#     tokenizer=tf.keras.preprocessing.text.Tokenizer()
    tokenizer.fit_on_texts([para])
    sequence = tokenizer.texts_to_sequences([para])
    sequence_length = 50  # Adjust as needed
    padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=sequence_length)
    tokenspara=tokenizenews(para)
    encoder_hidden_states=[model.predict(tokenspara)]
    summary=[]
    sequence_length = 50
    hidden_size = 128
    embedding_dim = 50
    context_vector=tf.zeros((128,50 ))
    decoder_state=tf.zeros((128, 50))
    current_token = np.zeros((128,))
    for j in range(length):
        input_sequence = tf.keras.Input(shape=(sequence_length, ))
        embedding_layer = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim)(input_sequence)
        model = tf.keras.Model(inputs=input_sequence, outputs=embedding_layer)
        output_embeddings = model.predict(current_token,verbose=0)
        attention_scores = []
        for i in range(len(encoder_hidden_states)):
            encoder_hidden_states[i] = tf.reshape(encoder_hidden_states[i], (128,50))
            e_t_i = v * tf.nn.tanh(Wh * encoder_hidden_states[i]+ Ws*decoder_state+  battn)
            attention_scores.append(e_t_i)
        attention_distributon=tf.nn.softmax(attention_scores)
        for i in range(len(encoder_hidden_states)):
            context_vector += attention_distributon[i] * encoder_hidden_states[i]
        output_embeddings=tf.reshape(output_embeddings, (128, 50, 1))
        output_embeddings=tf.squeeze(output_embeddings, axis=-1)
        concatenated_input = tf.expand_dims(tf.concat([context_vector,output_embeddings], axis=-1),axis=0)
        output_embeddings1=tf.matmul(concatenated_input,weights)+bias
        vocab_distribution=tf.nn.softmax(output_embeddings1,axis=1)
        highest_prob_index=np.argmax(vocab_distribution)
        current_token[j]=highest_prob_index
        decoder_state=(decoder_state+context_vector)/2
        decoder_state = tf.reshape(decoder_state, (128, 50))
        summary.append(current_token)
    tokentoword={tokenizer.word_index[i]:i for i in tokenizer.word_index.keys()}
    tokentoword.update({0.0:"",127.0:""})
    return (" ".join([tokentoword[i] for i in current_token]))

In [9]:
testarticle=pd.read_csv("/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/test.csv")
testarticle1=testarticle['article'][5:10].replace("'", "")
for i in testarticle1:
    print(i+'\n')
    print(tests(i,v,Ws,Wh,battn,weights,model,bias,tokenizer,100)+'\n')

This is the moment that a crew of firefighters struggled to haul a giant  pig out of a garden swimming pool. The prize porker, known  as Pigwig, had fallen into the pool in an upmarket neighbourhood in Ringwood, Hampshire. His owners had been taking him for a walk around the garden when the animal plunged into the water and was unable to get out. A team from Dorset Fire and Rescue struggled to haul the huge black pig out of swimming pool water . The prize porker known as Pigwig had fallen into the water and had then been unable to get out again . Two fire crews and a specialist animal rescue team had to use slide boards and strops to haul the huge black pig from the small pool. A spokesman for Dorset Fire and Rescue Service said: 'At 4.50pm yesterday the service received a call to a pig stuck in a swimming pool. 'One crew of firefighters from Ferndown and a specialist animal rescue unit from Poole were mobilised to this incident. 'Once in attendance the crew secured the pig with strops